In [0]:
%sql
CREATE CATALOG IF NOT EXISTS kaggle_test;
USE CATALOG kaggle_test;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS bronze;
USE SCHEMA bronze;

In [0]:
%python
# Use Managed Identity for authentication
spark.conf.set("fs.azure.account.auth.type.jamesbdatabricksdev.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.jamesbdatabricksdev.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ManagedIdentityTokenProvider")
spark.conf.set("fs.azure.account.oauth2.msi.client.id.jamesbdatabricksdev.dfs.core.windows.net", "jamesbdbconnector")

In [0]:

csv_path = "abfss://kaggle-test@jamesbdatabricksdev.dfs.core.windows.net/storagepath/bronze/olympics.csv"


df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(csv_path)

df.show()

In [0]:
%sql
  SELECT * FROM read_files(
  'abfss://kaggle-test@jamesbdatabricksdev.dfs.core.windows.net/storagepath/bronze',
  format => 'csv',
  header => true,
  mode => 'FAILFAST')
  LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TABLE olympics
AS
SELECT * FROM read_files(
  'abfss://kaggle-test@jamesbdatabricksdev.dfs.core.windows.net/storagepath/bronze',
  format => 'csv',
  header => true,
  mode => 'FAILFAST')

In [0]:
%sql
DESCRIBE olympics;

In [0]:
%sql
SELECT * from olympics LIMIT 10;

In [0]:
df = spark.read.format('csv') \
    .option('sep', ',') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .load('abfss://kaggle-test@jamesbdatabricksdev.dfs.core.windows.net/storagepath/bronze/stolen_vehicles.csv')

df.createOrReplaceTempView('stolen_vehicles')

display(df)

In [0]:
%sql
SELECT * FROM stolen_vehicles LIMIT 10;

In [0]:
%sql
DESCRIBE EXTENDED olympics;

In [0]:
%sql
DESCRIBE CATALOG kaggle_test;

In [0]:
%sql
DESCRIBE SCHEMA bronze;

In [0]:
%sql
DROP TABLE IF EXISTS stolen_vehicles_copy;
CREATE TABLE stolen_vehicles_copy;

In [0]:
%sql
COPY INTO stolen_vehicles_copy
FROM 'abfss://kaggle-test@jamesbdatabricksdev.dfs.core.windows.net/storagepath/bronze/stolen_vehicles.csv'
FILEFORMAT = csv
COPY_OPTIONS ('mergeSchema' = 'true','header'='true');
    
SELECT * FROM stolen_vehicles_copy LIMIT 10;

In [0]:
%sql
DROP TABLE olympics_improved;
CREATE TABLE olympics_improved;

In [0]:
%sql
COPY INTO olympics_improved
FROM 
(SELECT *,
current_timestamp() AS updated,
SPLIT(_metadata.file_path, '/')[SIZE(SPLIT(_metadata.file_path, '/')) - 1] AS source
FROM
'abfss://kaggle-test@jamesbdatabricksdev.dfs.core.windows.net/storagepath/bronze/stolen_vehicles.csv')
FILEFORMAT = csv
COPY_OPTIONS ('mergeSchema' = 'true','header'='true');

In [0]:
%sql
SELECT * from olympics_improved LIMIT 10;

In [0]:
%python
import sys
print(sys.version)